In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
dfs = pd.read_excel('S_ESTS.xlsx',header=None,
                    names=['CNUM','ICPSR','STATE','DISTRICT','STATENAME','PARTY','NAME','DIM1','DIM2','1','2','3','4','5','6','7'])
dfs2 = pd.read_excel('H_ESTS.xlsx',header=None,
                    names=['CNUM','ICPSR','STATE','DISTRICT','STATENAME','PARTY','NAME','DIM1','DIM2','1','2','3','4','5','6','7'])
dfs = dfs.append(dfs2)
dfs = dfs[dfs['CNUM'] >= 110]
print(dfs.shape)
# dfr = pd.read_csv('processed_recips_small.csv')
# dfr['bonica.rid'] = [t.lstrip('cand') for t in dfr['bonica.rid']]
dfr = pd.read_csv('DIME_RECIP.csv')
dfr = dfr[np.logical_and(dfr['cycle'] >= 2006,dfr['cycle'] <= 2012)]
temp = dfr['ICPSR2'].values
temp2 = [t[-5:] for t in temp]
temp2 = [re.sub("\D","",s) for s in temp2]
# temp2 = [t.lstrip('0') for t in temp2]
dfr['ICPSR2'] = temp2
dfr['bonica.rid'] = [t.lstrip('cand') for t in dfr['bonica.rid']]
rid_dict = {}
for i,row in dfr.iterrows():
    if row['ICPSR2'] != '':
        rid_dict[int(row['ICPSR2'])] = row['bonica.rid']
columns = list(dfs.columns) + list(['RID'])
dfNew = pd.DataFrame()
for i,row in dfs.iterrows():
    if int(row['ICPSR']) in rid_dict.keys():
        dfNew = dfNew.append(pd.Series(np.append(row.values,[rid_dict[row['ICPSR']]]).T),ignore_index=True)

(2197, 16)


C:\Users\jerem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,4,5,11,12,13,14,15,16,17,18,21,22,36,54,55,56,57,58,59,62,64,66,67,68,69,70,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
dfc = pd.read_csv('used_contributors_small.csv')
dfNew.columns = columns
dfNew = dfNew.reindex(columns = list(columns) + list(dfc['bonica.cid'].values.astype(int)))
bid = dfc['bonica.cid'].values.astype(int)
dfNew['RID'] = [t.lstrip('omm') for t in dfNew['RID']]

C:\Users\jerem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
dfNew = dfNew.fillna(value=0)
del(dfr)
del(dfs)
del(dfc)
dfNew = dfNew.T

In [ ]:
dfc = pd.read_csv('used_contributors_small.csv')
bid = dfc['bonica.cid'].values.astype(int)
del(dfc)
dfNew = pd.read_csv('finallyProcessed2.csv')
filenames = ['contribDB_2006.csv','contribDB_2008.csv','contribDB_2010.csv','contribDB_2012.csv']
cnums = [110,111,112,113]
count = 0
# for k in range(4):
for temp in pd.read_csv(filenames[3],encoding = "ISO-8859-1", chunksize=10000000):
#     temp = pd.read_csv(filenames[2],encoding = "ISO-8859-1")
    temp['bonica.rid'] = [re.sub("\D","",s) for s in temp['bonica.rid'].astype(str)]
    for i,row in dfNew.iterrows():
        if i%100 == 0:
            print(i)
        if row['CNUM'] == cnums[3]:
            temp2 = temp[temp['bonica.rid'] == row['RID']]
            for j,row2 in temp2.iterrows():
                if row2['bonica.cid'] in bid:
                    count += 1
                    if count%1000 == 0:
                        print(count)
                    dfNew[row2['bonica.cid']].iloc[i] = dfNew[row2['bonica.cid']].iloc[i] + row2['amount']
    del(temp)

C:\Users\jerem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dfNew.to_csv('finallyProcessed2.csv')

In [51]:
df = pd.read_csv('processed_recips_small.csv')

In [13]:
df.columns

Index(['Unnamed: 0', 'index', 'election', 'cycle', 'fecyear', 'Cand.ID',
       'FEC.ID', 'NID', 'ICPSR', 'ICPSR2', 'bonica.rid', 'bonica.cid', 'name',
       'lname', 'ffname', 'fname', 'mname', 'nname', 'title', 'suffix',
       'party', 'state', 'seat', 'district', 'Incum.Chall',
       'recipient.cfscore', 'contributor.cfscore', 'recipient.cfscore.dyn',
       'dwnom1', 'dwnom2', 'ps.dwnom1', 'ps.dwnom2', 'dwdime', 'irt.cfscore',
       'num.givers', 'num.givers.total', 'n.data.points.personal.donations',
       'n.data.points.personal.donations.unq', 'cand.gender',
       'total.disbursements', 'total.pc.contribs', 'contribs.from.candidate',
       'unitemized', 'non.party.ind.exp.for', 'non.party.ind.exp.against',
       'ind.exp.for', 'ind.exp.against', 'comm.cost.for', 'comm.cost.against',
       'party.coord.exp', 'party.ind.exp.against', 'total.receipts',
       'total.indiv.contrib', 'total.pac.contribs', 'ran.primary',
       'ran.general', 'p.elec.stat', 's.elec.stat', 'r.

In [52]:
df = df.drop(['Unnamed: 0'],axis=1)

In [53]:
df = df.T

In [18]:
dfc = pd.read_csv('used_contributors_small.csv')

C:\Users\jerem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ValueError: Length mismatch: Expected axis has 75 elements, new values have 200538 elements

In [54]:
for i, col in df.iteritems():
    if col.loc['cycle'] == 2012:
        df = df.drop(i,axis=1)

In [ ]:
z = np.zeros(df.shape[1])
t = pd.Series(list(z))
for s in dfc['bonica.cid']:
    t.name = s
    df = df.append(t)

In [38]:
df.shape

(75, 2665)

In [50]:
df

,0,1,2,3,4,5,6,7,8,9,...,2655,2656,2657,2658,2659,2660,2661,2662,2663,2664
index,7461,57407,57448,57483,57516,57942,58524,58525,58526,58527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
election,2010,fdfd2006,fdfd2006,fdfd2006,fdfd2006,fdfd2006,fdfd2006,fdfd2006,fdfd2006,fdfd2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cycle,2010,2006,2006,2006,2006,2006,2006,2006,2006,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fecyear,2010,2004,1998,2006,2006,2006,2006,2006,2006,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cand.ID,VA_CC982850,H8OH09046,H8CA36089,H6WA09041,H6VA09052,H6NJ13225,H6CA36141,H6CA36133,H6CA36125,H6CA36117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FEC.ID,NaN,C00334839,C00333716,C00415497,C00418434,H6NJ13225,C00429803,H6CA36133,C00426379,C00493585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NID,NaN,N00003520,N00006808,N00027715,N00027970,N00000680,N00028388,N00006673,N00028202,N00028125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ICPSR,VA_CC9828502010,H8OH090462006,H8CA360892006,H6WA090412006,H6VA090522006,H6NJ132252006,H6CA361412006,H6CA361332006,H6CA361252006,H6CA361172006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ICPSR2,9033,9046,36089,9041,9052,9058,36141,36133,36125,36117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bonica.rid,cand113825,cand56017,cand53726,cand53169,cand53071,cand51728,cand49679,cand49678,cand49677,cand49676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
dfc

,Unnamed: 0,bonica.cid,contributor.type,num.records,num.distinct,most.recent.contributor.name,most.recent.contributor.address,most.recent.contributor.city,most.recent.contributor.zipcode,most.recent.contributor.state,...,amount_1996,amount_1998,amount_2000,amount_2002,amount_2004,amount_2006,amount_2008,amount_2010,amount_2012,amount_2014
0,10,1.003300e+04,C,1631,194.0,montana democratic party non federal account,po box 802,helena,596240802,MT,...,138276.0,268345.0,650604.0,369966.0,12333.0,198005.0,151171.0,30489.0,81145.0,10000.0
1,25,1.004040e+05,C,10051,1354.0,napus pac for postmasters,8 herbert st,alexandria,223052600,VA,...,420848.0,399829.0,419935.0,611900.0,599752.0,649778.0,555310.0,643360.0,470924.0,169565.0
2,39,1.005370e+05,C,466,305.0,bill thomas campaign committee,po box 395,bakersfield,933020395,CA,...,180499.0,550500.0,297500.0,1055950.0,514979.0,494423.0,0.0,0.0,0.0,0.0
3,100,1.012120e+05,C,3164,123.0,right to life of michigan political action com...,po box 901,grand rapids,495090901,MI,...,169824.0,3743.0,372675.0,29507.0,348113.0,124217.0,11783.0,12321.0,219685.0,0.0
4,114,1.016000e+03,C,13869,2141.0,carpenters legislative improvement committee u...,"101 constiution ave, nw",washington,20001,DC,...,1531850.0,1452923.0,1831700.0,2260000.0,2252000.0,2906000.0,2444349.0,2522000.0,1934500.0,20000.0
5,115,1.017660e+05,C,3849,849.0,continental airlines inc employee fund for a b...,1600 smith st 15th fl hqseo,houston,770027362,TX,...,63750.0,149600.0,179163.0,275679.0,257529.0,443330.0,470780.0,492676.0,529021.0,64000.0
6,135,1.023680e+05,C,1044,380.0,the desert caucus,po box 44146,tucson,857334146,AZ,...,148750.0,168000.0,151500.0,158000.0,145500.0,192000.0,180000.0,155000.0,325000.0,30000.0
7,159,1.032200e+04,C,15773,1863.0,apwu,"1300 l st, nw",washington,20005,DC,...,981610.0,819240.0,1032601.0,605002.0,2255500.0,1973050.0,1980629.0,2260782.0,3771024.0,1146800.0
8,163,1.035490e+05,C,3261,564.0,parsons corporation political action committee,100 w walnut st t 1110,pasadena,911240001,CA,...,72400.0,102750.0,77050.0,237500.0,333100.0,564125.0,612000.0,630888.0,842600.0,284000.0
9,164,1.035645e+09,I,72,22.0,"hanes, william lee",607 green acres,solon,52333,IA,...,0.0,0.0,0.0,0.0,0.0,20.0,260.0,575.0,4931.0,1590.0
